# Workflow notebook for the PolymerXtal SimTool #
### Tongtong Shen, Jessica Nash and Alejandro Strachan ###
### School of Materials Engineering, Purdue University and The Molecular Sciences Software Institute###

The PolymerXtal SimTool is based on PolymerXtal software designed to build and analyze molecular-level polymer crystal structures. PolymerXtal provides a standardized process to generate polymer crystal structure based on monomer, tacticity, helicity, chiriality and unit cell information and analyze the crystallinity in polymer systems with given atom trajectories. These features have allowed PolymerXtal to lead further investigations of semi-crystalline polymers where most properties are governed by the crystalline microstructure and promote future research endeavors in the area of crystalline polymer simulations.

In general, users need to specify as inputs the polymer type they wish to run, the helice structure, tacticity and chiriality. The SimTool generates the chain helix structure and a visual snapshot of it.

The goal of the workflow notebook is to set up and run the PolymerXtal SimTool, which is demonstrated both for the case a single build, or as a high-throughput study, exploring various polymer types and helicity at once.

## Set up imports ##

We will first import the necessary libraries and packages, including the simtool package which helps locate the SimTool we wish to run and pass it inputs and outputs

In [1]:
from hublib.tool import get_inputs, run_simtool, rdisplay, read, get_outputs_df, get_outputs, get_output_files
from simtool import findInstalledSimToolNotebooks, searchForSimTool, findSimToolNotebook
from simtool import getSimToolInputs, getSimToolOutputs, Run

import os

## SimTool information ##

We will first create a dictionary containing the SimTool information using the "findSimToolNotebook" function

In [2]:
# Identify the simtool and retrieve its status
simToolName = "polymerxtal"
nb = searchForSimTool('polymerxtal')
for key in nb.keys():
    print("%18s = %s" % (key,nb[key]))

      notebookPath = /home/nanohub/shen276/polymerxtal/simtool/polymerxtal.ipynb
       simToolName = polymerxtal
   simToolRevision = None
         published = False


### SimTool Inputs and Outputs ###
We will then create an object for the simtool inputs using the "getSimToolInputs" function - this object is validated by the simtool library so only allowed values are passed to the simtool

In [3]:
# get the list of inputs
inputs = getSimToolInputs(nb)
print(inputs)

polymer_type:
    type: Text
    description: Select the polymer type.
    options: ['PAN', 'PE', 'PP', 'PS', 'POM', 'PTFE', 'PVC']
    value: PAN

helice_atoms:
    type: Integer
    description: Number of chain (backbone) atoms of a motif of the helix
    min: 2
    max: 2
    value: 2

helice_motifs:
    type: Integer
    description: Number of motifs of the helix
    min: 1
    max: 3
    value: 3

helice_turns:
    type: Integer
    description: Number of motifs of the helix
    min: 1
    max: 1
    value: 1

monomers:
    type: Number
    description: Number of monomers
    min: 1
    max: 5000
    value: 40

tacticity:
    type: Text
    description: Select the tacticity of the polymer chain.
    options: ['isotactic', 'atactic', 'syndiotactic', 'None']
    value: isotactic

chiriality:
    type: Text
    description: Select the chiriality of the polymer chain.
    options: ['left', 'right', 'None']
    value: right

head_tail_defect_ratio:
    type: Number
    description: Rat

In [4]:
# we can get a list of expected outputs
getSimToolOutputs(nb)

Helix Name:
    type: Text

Output PDBfile:
    type: Text

Output PDB:
    type: Text

## Setting SimTool Inputs ##

Now we will setup the inputs and run a first simulation. Important aspects on SimTool inputs are:
 * Inputs are validated

In [5]:
# range checking
inputs.monomers.value = 6000

ValueError: Maximum value is 5000

## Run the SimTool ###

In [6]:
inputs.polymer_type.value = 'PE'

In [7]:
r = Run(nb, inputs)

runname   = 36ea9e1e00de46f4a8a00f88786fb3b9
outdir    = RUNS/36ea9e1e00de46f4a8a00f88786fb3b9
cached    = False
published = False
submit --local papermill -f inputs.yaml
       /home/nanohub/shen276/polymerxtal/simtool/polymerxtal.ipynb
       polymerxtal.ipynb


Input Notebook:  /home/nanohub/shen276/polymerxtal/simtool/polymerxtal.ipynb
Output Notebook: polymerxtal.ipynb
Executing: 100%|██████████| 13/13 [00:26<00:00,  2.05s/cell]


## Explore PolymerXtal outputs ##
The Run object contains a useful method called "getResultSummary", which we can use to view the SimTool outputs at a high level in a pandas dataframe.

In [8]:
r.getResultSummary()

,name,data,encoder,display,filename
0,simToolSaveErrorOccurred,0,text,None,polymerxtal.ipynb
1,simToolAllOutputsSaved,1,text,None,polymerxtal.ipynb
2,Helix Name,"""PE_helix_2_3_1_iso+""",text,None,polymerxtal.ipynb
3,Output PDBfile,file://PE_helix_2_3_1_iso+.pdb,text,None,polymerxtal.ipynb
4,Output PDB,"""ATOM 1 H UNK A 1 2.328 1.95...",text,None,polymerxtal.ipynb


In [9]:
# Let's look at the final structure first
pdbFile = r.read('Output PDBfile',raw=True)
import imolecule
imolecule.draw(pdbFile[7:],size=(1000, 300))
#imolecule.draw(r.outdir+"/"+r.read('Helix Name')+".pdb",size=(1000, 300))#, camera_type="orthographic") 

## Scripting multiple PolymerXtal builds ##

The above cells demonstrated a single build using the simtool library. We will now demonstrate a high-throughput approach to setting up multiple builds, where will demonstrated various polymer types and helices.

In [10]:
polymer_type_list = ['PP', 'PS']

for polymer_type in polymer_type_list:
    print ("Polymer Type: ", polymer_type)
   
    inputs.polymer_type.value = polymer_type
    
    # run the SimTool with the input values
    r = Run(nb, inputs)
    

Polymer Type:  PP
runname   = bc1c4512394c47019ef0466454b5869e
outdir    = RUNS/bc1c4512394c47019ef0466454b5869e
cached    = False
published = False
submit --local papermill -f inputs.yaml
       /home/nanohub/shen276/polymerxtal/simtool/polymerxtal.ipynb
       polymerxtal.ipynb


Input Notebook:  /home/nanohub/shen276/polymerxtal/simtool/polymerxtal.ipynb
Output Notebook: polymerxtal.ipynb
Executing: 100%|██████████| 13/13 [00:51<00:00,  3.99s/cell]


Polymer Type:  PS
runname   = 93d8b3a50b7246ba85c4d7cd6bf1c569
outdir    = RUNS/93d8b3a50b7246ba85c4d7cd6bf1c569
cached    = False
published = False
submit --local papermill -f inputs.yaml
       /home/nanohub/shen276/polymerxtal/simtool/polymerxtal.ipynb
       polymerxtal.ipynb


Input Notebook:  /home/nanohub/shen276/polymerxtal/simtool/polymerxtal.ipynb
Output Notebook: polymerxtal.ipynb
Executing: 100%|██████████| 13/13 [03:23<00:00, 15.69s/cell]


In [11]:
#visualize final snapshot of structure
pdbFile = r.read('Output PDBfile',raw=True)
imolecule.draw(pdbFile[7:],size=(1000, 300))
#imolecule.draw(r.outdir+"/"+r.read('Helix Name')+".pdb",size=(1000, 300))#, camera_type="orthographic") 